# Barcelona test

## Preparations

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import esda
from splot.esda import lisa_cluster, moran_scatterplot
from libpysal import graph
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn import cluster

In [ ]:
link = 'https://opendata-ajuntament.barcelona.cat/data/dataset/d8e40c96-9f1f-4fd3-86da-2baa1599616d/resource/edaf6642-a51b-4b2b-a492-fa913d5e8b91/download/2021_atles_renda_bruta_llar.csv'
gross_income = gpd.read_file(link)

link1 = 'https://opendata-ajuntament.barcelona.cat/data/dataset/147d27cf-13a0-49e4-812e-1decc3cb4736/resource/2405d999-5755-462b-b1e9-7e7dafacb384/download/2021_atles_renda_mediana.csv'
equival_income = gpd.read_file(link1)

children = gpd.read_file('./data/2021_pad_dom_mdbas_edat-0018.csv')

edu = gpd.read_file('./data/2021_pad_mdbas_niv-educa-esta_sexe.csv')

seccio = gpd.read_file('./data/seccio_censal')

### Data Wrangling

In [ ]:
children = children.drop(
    columns=['Data_Referencia',
             'Codi_Barri',
             'Nom_Barri',
             'AEB',
             'geometry',
             'Nom_Districte']
             ).rename(columns={'Codi_Districte': 'district_code',
                               'Seccio_Censal': 'section_code',
                               'Valor': 'households',
                               'DOM_00_18': 'children'
                               }
                               )
children['households'] = children['households'].astype(int)
children['children'] = children['children'].astype(int)

children['district_code_length'] = children['district_code'].str.len()
children['section_code'] = children.apply(
    lambda row: row['section_code'][row['district_code_length']:], axis=1
    )
children = children.drop(columns=['district_code_length'])

children['section_code'] = children['section_code'].astype(int)
children['district_code'] = children['district_code'].astype(int)

children['weighted_children'] = children['children'] * children['households']
grouped = children.groupby(['district_code', 'section_code']).agg(
    total_households=('households', 'sum'),
    total_weighted_children=('weighted_children', 'sum')
)
grouped['average_children'] =(
    grouped['total_weighted_children'] / grouped['total_households']
    )
children = grouped.reset_index().drop(
    columns=['total_weighted_children',
             'total_households'
             ]
             )

children.head()

In [ ]:
equival_income = equival_income.drop(
    columns=['Any', 'Codi_Barri', 'Nom_Barri', 'geometry', 'Nom_Districte']
    ).rename(columns={'Mediana_Renda_€': 'equival_income',
                      'Codi_Districte': 'district_code',
                      'Seccio_Censal': 'section_code'
                      }
                      )
equival_income['equival_income'] = equival_income['equival_income'].astype(int)
equival_income['district_code'] = equival_income['district_code'].astype(int)
equival_income['section_code'] = equival_income['section_code'].astype(int)
equival_income.info()

In [ ]:
gross_income = gross_income.drop(
    columns=['Any', 'Codi_Barri', 'Nom_Barri', 'geometry']
    ).rename(columns={'Import_Renda_Bruta_€': 'gross_income',
                      'Codi_Districte': 'district_code',
                      'Nom_Districte': 'district_name',
                      'Seccio_Censal': 'section_code',
                      }
                      )
gross_income['section_code'] = gross_income['section_code'].astype(int)
gross_income['district_code'] = gross_income['district_code'].astype(int)
gross_income['gross_income'] = gross_income['gross_income'].astype(float)
gross_income.info()

In [ ]:
edu = edu.drop(
    columns=['Data_Referencia',
             'Codi_Barri',
             'Nom_Barri',
             'AEB',
             'geometry',
             'Nom_Districte']
             ).rename(columns={'Codi_Districte': 'district_code',
                               'Seccio_Censal': 'section_code',
                               'Valor': 'people',
                               'NIV_EDUCA_esta': 'education_level',
                               'SEXE': 'gender'
                               }
                               )

edu['district_code_length'] = edu['district_code'].str.len()
edu['section_code'] = edu.apply(
    lambda row: row['section_code'][row['district_code_length']:], axis=1
    )
edu = edu.drop(columns=['district_code_length'])

edu['section_code'] = edu['section_code'].astype(int)
edu['district_code'] = edu['district_code'].astype(int)
edu['people'] = edu['people'].replace('..', np.nan).astype(float)


grouped = edu.groupby([
    'district_code',
    'section_code',
    'education_level',
    'gender'
    ])['people'].sum()

unstacked = grouped.unstack(level=[2, 3])
unstacked.reset_index(inplace=True)
unstacked.fillna(0, inplace=True)

unstacked.columns = ['_'.join(col) for col in unstacked.columns.values]

column_mapping = {
    col: (
        col.replace('_1', '_f')
        .replace('_2', '_m')
        .replace('district_code_', 'district_code')
        .replace('section_code_', 'section_code')
        .replace('1_', 'no_edu_')
        .replace('2_', 'prim_')
        .replace('5_', 'high_')
        .replace('6_', 'no_data_')
        )
        for col in unstacked.columns
}

unstacked.rename(columns=column_mapping, inplace=True)

unstacked['sec_f'] = unstacked['3_f'] + unstacked['4_f']
unstacked['sec_m'] = unstacked['3_m'] + unstacked['4_m']

unstacked = unstacked.drop(columns=['3_f', '4_f', '3_m', '4_m'])

columns_to_move = ['sec_f', 'sec_m']
new_position = 7

original_columns = unstacked.columns.to_list()

for column in columns_to_move:
    original_columns.remove(column)

for column in columns_to_move:
    original_columns.insert(new_position -1, column)
    new_position += 1

edu = unstacked[original_columns]

edu.head()

In [ ]:
seccio = seccio[['DISTRICTE', 'SEC_CENS', 'geometry']]
seccio = seccio.rename(columns={
    'DISTRICTE': 'district_code',
    'SEC_CENS': 'section_code'
    }
    )

seccio['district_code'] = seccio['district_code'].astype(int)
seccio['section_code'] = seccio['section_code'].astype(int)

seccio.info()

#### Merging

In [ ]:
for df in [children, equival_income, gross_income, edu]:
    seccio = seccio.merge(df, on=['district_code', 'section_code'])

seccio.head()

## Exploration

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

seccio.plot(column='average_children', ax=ax[0], legend=True, cmap='magma')
seccio.plot(column='equival_income', ax=ax[1], legend=True, cmap='magma')
seccio.plot(column='gross_income', ax=ax[2], legend=True, cmap='magma')

ax[0].set_title('Average children per household')
ax[1].set_title('Equivalent income')
ax[2].set_title('Gross income')

ax[0].tick_params(axis='x', rotation=45)
ax[1].tick_params(axis='x', rotation=45)
ax[2].tick_params(axis='x', rotation=45)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(10,17))

seccio.plot(column='no_edu_f', ax=ax[0, 0], legend=True, cmap='magma')
seccio.plot(column='no_edu_m', ax=ax[0, 1], legend=True, cmap='magma')
seccio.plot(column='prim_f', ax=ax[1, 0], legend=True, cmap='magma')
seccio.plot(column='prim_m', ax=ax[1, 1], legend=True, cmap='magma')
seccio.plot(column='sec_f', ax=ax[2, 0], legend=True, cmap='magma')
seccio.plot(column='sec_m', ax=ax[2, 1], legend=True, cmap='magma')
seccio.plot(column='high_f', ax=ax[3, 0], legend=True, cmap='magma')
seccio.plot(column='high_m', ax=ax[3, 1], legend=True, cmap='magma')

ax[0, 0].set_title('No education, women')
ax[0, 1].set_title('No education, men')
ax[1, 0].set_title('Primary education, women')
ax[1, 1].set_title('Primary education, men')
ax[2, 0].set_title('Secondary education, women')
ax[2, 1].set_title('Secondary education, men')
ax[3, 0].set_title('Higher education, women')
ax[3, 1].set_title('Higher education, men')

ax[0, 0].tick_params(axis='x', rotation=45)
ax[0, 1].tick_params(axis='x', rotation=45)
ax[1, 0].tick_params(axis='x', rotation=45)
ax[1, 1].tick_params(axis='x', rotation=45)
ax[2, 0].tick_params(axis='x', rotation=45)
ax[2, 1].tick_params(axis='x', rotation=45)
ax[3, 0].tick_params(axis='x', rotation=45)
ax[3, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()

### LISA

In [ ]:
contiguity = graph.Graph.build_contiguity(seccio, rook=False)
contiguity_r = contiguity.transform("r")

lisa = esda.Moran_Local(seccio['gross_income'], contiguity_r.to_W())

In [ ]:
_ = lisa_cluster(lisa, seccio)

In [ ]:
_ = moran_scatterplot(lisa, p=0.05, scatter_kwds={"s": 5, "alpha":.2})

## Clustering

### Preparation

In [ ]:
subranks = [
    'average_children', 'gross_income',
    'high_f', 'sec_f', 'prim_f', 'no_edu_f',
    ]

scaler = StandardScaler()
data_scaled = scaler.fit_transform(seccio[subranks])

scores = []
for k in range(2, 11):
    kmeans = cluster.KMeans(n_clusters=k)
    kmeans.fit(data_scaled)
    scores.append(silhouette_score(data_scaled, kmeans.labels_))

optimal_clusters = scores.index(max(scores)) + 2

plt.plot(range(2, 11), scores)
plt.title('Silhouette Score vs Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

#### KMeans

In [ ]:
kmeans3 = cluster.KMeans(n_clusters=optimal_clusters, random_state=42)
kmeans3.fit(seccio[subranks])
seccio['kmeans3'] = kmeans3.labels_

#### Spatially-lagged cluster

In [ ]:
queen = graph.Graph.build_contiguity(seccio)
queen_r = queen.transform("r")

for column in subranks:
    seccio[column + "_lag"] = queen_r.lag(seccio[column])

subranks_lag = [column + "_lag" for column in subranks]
subranks_spatial = subranks + subranks_lag

kmeans3_lag = cluster.KMeans(n_clusters=3, random_state=42)
kmeans3_lag.fit(seccio[subranks_spatial])
seccio['kmeans3_lag'] = kmeans3_lag.labels_

#### Regionalisation

In [ ]:
agg3 = cluster.AgglomerativeClustering(n_clusters=3, connectivity=queen.sparse)
agg3.fit(seccio[subranks])
seccio['agg3'] = agg3.labels_

### Visualisation

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 18))
seccio.plot(column='kmeans3', ax=ax[0], legend=True, categorical=True, cmap='tab20b')
seccio.plot(column='kmeans3_lag', ax=ax[1], legend=True, categorical=True, cmap='tab20b')
seccio.plot(column='agg3', ax=ax[2], legend=True, categorical=True, cmap='tab20b')

ax[0].set_title('kmeans3')
ax[1].set_title('kmeans3_lag')
ax[2].set_title('agg3')

ax[0].tick_params(axis='x', rotation=45)
ax[1].tick_params(axis='x', rotation=45)
ax[2].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()